# Lựa Chọn Thuật Toán

Notebook này so sánh các lựa chọn thuật toán cho 2 bài toán: recommendation và clustering.


## Recommendation System

### Ứng viên
- K-Nearest Neighbors dựa trên vector đặc trưng (có thể dùng cosine similarity).
- Matrix Factorization (SVD) nếu có dữ liệu tương tác người dùng.
- Content-based TF-IDF/embedding từ mô tả văn bản.

### Tiêu chí chọn
- Dữ liệu hiện tại chủ yếu là thông tin nội dung => ưu tiên content-based KNN + cosine.
- Có thể mở rộng bằng sentence embedding (SBERT) khi cần.

## Clustering

### Ứng viên
- KMeans: dễ triển khai, phù hợp dữ liệu numeric đã chuẩn hóa.
- Gaussian Mixture Model: mô hình phân phối mềm.
- HDBSCAN/DBSCAN: nếu muốn cụm không cầu tròn.

### Tiêu chí chọn
- Dữ liệu đã chuẩn hóa => KMeans cho baseline.
- Xem xét silhouette score để chọn số cụm.
- Có thể bổ sung HDBSCAN để phát hiện outlier.


In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

DATA_PATH = "/Users/doananh/Documents/Documents - Doan’s MacBook Pro/đồ án DS/project2/data_motobikes.xlsx - Sheet1.csv"

df = pd.read_csv(DATA_PATH)

features = df[[
    "Giá", "Khoảng giá min", "Khoảng giá max", "Năm đăng ký", "Số Km đã đi",
    "Thương hiệu", "Loại xe", "Dung tích xe", "Tình trạng"
]].copy()

# Làm sạch các cột số dạng chuỗi
price_cols = ["Giá", "Khoảng giá min", "Khoảng giá max"]
for col in price_cols:
    features[col] = (
        features[col]
        .astype(str)
        .str.lower()
        .str.replace("đ", "", regex=False)
        .str.replace("vnđ", "", regex=False)
        .str.replace("vnd", "", regex=False)
        .str.replace("triệu", "", regex=False)
        .str.replace("tr", "", regex=False)
        .str.replace("ty", "", regex=False)
        .str.replace(",", "", regex=False)
        .str.replace(".", "", regex=False)
        .str.extract(r"(\d+)")
        .astype(float)
        / 1_000_000
    )

features["Năm đăng ký"] = pd.to_numeric(features["Năm đăng ký"], errors="coerce")
features["Số Km đã đi"] = (
    features["Số Km đã đi"].astype(str)
    .str.lower()
    .str.replace("km", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.replace(".", "", regex=False)
)
features["Số Km đã đi"] = pd.to_numeric(features["Số Km đã đi"], errors="coerce")

numeric_cols = ["Giá", "Khoảng giá min", "Khoảng giá max", "Năm đăng ký", "Số Km đã đi"]
cat_cols = ["Thương hiệu", "Loại xe", "Dung tích xe", "Tình trạng"]

required_cols = numeric_cols + cat_cols

df_model = features.dropna(subset=required_cols)

preprocess = ColumnTransformer([
    ("num", StandardScaler(), numeric_cols),
    (
        "cat",
        OneHotEncoder(handle_unknown="ignore", sparse_output=False),
        cat_cols,
    ),
])

X = preprocess.fit_transform(df_model)

results = []
for k in range(3, 9):
    km = KMeans(n_clusters=k, random_state=42, n_init="auto")
    labels = km.fit_predict(X)
    score = silhouette_score(X, labels)
    results.append({"k": k, "silhouette": score})

pd.DataFrame(results)


,k,silhouette
0,3,0.191928
1,4,0.192171
2,5,0.289678
3,6,0.218760
4,7,0.193475
5,8,0.222263


Bảng kết quả giúp xác định giá trị `k` với silhouette cao nhất. Có thể trực quan hóa thêm bằng line chart.
